Muhammad Nabil Bin Muhamad - IS01082117
Ahmad Amirul Aizad Bin Rosmadi - IS01082507

In [93]:
import pandas as pd
import numpy as np
import re
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [95]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\victus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\victus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\victus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\victus\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [97]:
df = pd.read_csv("Reviews.csv")

In [99]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [101]:
df = df[['Score', 'Text']]

In [103]:
df.dropna(inplace=True)

In [105]:
df = df[df['Score'] != 3]

In [107]:
df['Sentiment'] = df['Score'].apply(lambda x: 1 if x > 3 else 0)

In [109]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
    text = re.sub(r'\d+', '', text)      # remove numbers
    return text

In [111]:
def tokenize_text(text):
    return word_tokenize(text)

# c. Remove stopwords
stop_words = set(stopwords.words('english'))
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

In [113]:
stemmer = PorterStemmer()
def stem_words(tokens):
    return [stemmer.stem(word) for word in tokens]

In [115]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(tokens):
    return [lemmatizer.lemmatize(word) for word in tokens]

In [117]:
def full_preprocess_pipeline(text):
    text = clean_text(text)
    tokens = tokenize_text(text)
    tokens = remove_stopwords(tokens)
    tokens = lemmatize_words(tokens)  # You can use stem_words(tokens) instead
    return " ".join(tokens)

In [119]:
df['Processed_Text'] = df['Text'].apply(full_preprocess_pipeline)

In [121]:
df[['Text', 'Processed_Text']].head()

,Text,Processed_Text
0,I have bought several of the Vitality canned d...,bought several vitality canned dog food produc...
1,Product arrived labeled as Jumbo Salted Peanut...,product arrived labeled jumbo salted peanutsth...
2,This is a confection that has been around a fe...,confection around century light pillowy citrus...
3,If you are looking for the secret ingredient i...,looking secret ingredient robitussin believe f...
4,Great taffy at a great price. There was a wid...,great taffy great price wide assortment yummy ...


In [123]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [125]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Split features and labels
X_raw = df['Processed_Text']   # text data
y = df['Sentiment']            # labels (0 = negative, 1 = positive)

In [127]:
# Initialize CountVectorizer (BoW)
bow_vectorizer = CountVectorizer()

# Transform text data into feature vectors
X_bow = bow_vectorizer.fit_transform(X_raw)

# View shape of feature matrix
print("BoW matrix shape:", X_bow.shape)

BoW matrix shape: (525814, 203766)


In [129]:
# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Transform text data into TF-IDF feature vectors
X_tfidf = tfidf_vectorizer.fit_transform(X_raw)

# View shape of feature matrix
print("TF-IDF matrix shape:", X_tfidf.shape)


TF-IDF matrix shape: (525814, 203766)


In [131]:
# Print sample feature names
print("Sample BoW Features:", bow_vectorizer.get_feature_names_out()[:10])
print("Sample TF-IDF Features:", tfidf_vectorizer.get_feature_names_out()[:10])

Sample BoW Features: ['__' '___' '____' '_____' '______' '_______' '________' '__________'
 '___________' '________________________________________']
Sample TF-IDF Features: ['__' '___' '____' '_____' '______' '_______' '________' '__________'
 '___________' '________________________________________']


In [133]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [135]:
# Split data into 80% train and 20% test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [137]:
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
y_pred_nb = nb_model.predict(X_test)

In [139]:
svm_model = LinearSVC()
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

In [143]:
def evaluate_model(y_true, y_pred, model_name):
    print(f"📊 Evaluation for {model_name}")
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
    print("Classification Report:\n", classification_report(y_true, y_pred))
    print("-" * 50)

# Evaluate each
evaluate_model(y_test, y_pred_nb, "Naive Bayes")
evaluate_model(y_test, y_pred_svm, "Support Vector Machine")

📊 Evaluation for Naive Bayes
Accuracy: 0.8567747211471716
Confusion Matrix:
 [[ 1412 14967]
 [   95 88689]]
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.09      0.16     16379
           1       0.86      1.00      0.92     88784

    accuracy                           0.86    105163
   macro avg       0.90      0.54      0.54    105163
weighted avg       0.87      0.86      0.80    105163

--------------------------------------------------
📊 Evaluation for Support Vector Machine
Accuracy: 0.9449711400397478
Confusion Matrix:
 [[12507  3872]
 [ 1915 86869]]
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.76      0.81     16379
           1       0.96      0.98      0.97     88784

    accuracy                           0.94    105163
   macro avg       0.91      0.87      0.89    105163
weighted avg       0.94      0.94      0.94    105163

--------------------

Name 1: Muhammad Nabil Bin Muhamad
Student ID 1: IS01082117

Name 2 : Ahmad Amirul Aizad Bin Rosmadi
Student ID 2 : IS01082507

In this lab assignment, for text pre-processing I (Nabil) and Aizad implemented 5 steps which are **1. Tokenization**, **2. Standardization**, **3. Stop Word Removal**, **4. Stemming**, **5. Lemmetization**. Then, we implemented and compared two machine learning models: **Multinomial Naive Bayes** and **Support Vector Machine (SVM)** for sentiment classification using the Amazon Fine Food Reviews dataset. I tried to include **Random Forest Regression** but our laptop takes time too long to run the Random Forest Regression so we give up. Sorry Madam Laila. Here are our findings :

**Naive Bayes** is a simple, fast, and highly scalable classifier, especially effective for text classification tasks with a large feature space. It performed well and trained quickly, making it ideal for large datasets. However, it assumes feature independence, which may reduce accuracy when the text contains complex dependencies between words.

**Support Vector Machine (SVM)**, on the other hand, produced higher accuracy and precision compared to Naive Bayes. SVM is powerful in handling high-dimensional data and works well with sparse features like TF-IDF. Its main downside is that it's computationally more intensive and slower to train, especially with large datasets.

In summary, **SVM outperformed Naive Bayes in terms of accuracy**, but **Naive Bayes offers efficiency and simplicity**. The choice of model depends on the task requirements—speed vs performance.

In [147]:
df.to_csv("preprocessed_reviews.csv", index=False)